# CDS services in Jupyter notebooks

Katarina A. Lutz¹

1. Université de Strasbourg, CNRS, Observatoire Astronomique de Strasbourg, UMR 7550, F-67000, Strasbourg, France

***



## Introduction

Welcome to a Jupyter notebook demonstrating how to access some of the services provided by the Strasbourg astronomical Data Center ([CDS](https://cdsweb.u-strasbg.fr/)) with Python.  

In this tutorial, we will:
1. gather information on a specific object using [![Simbad](https://custom-icon-badges.demolab.com/badge/Simbad-gray.svg?logo=simbad&logoColor=lightblue&logoWidth=20)](https://simbad.u-strasbg.fr/simbad/ "https://simbad.u-strasbg.fr/simbad/"), 
2. visualise it through survey images using the [![Aladin](https://custom-icon-badges.demolab.com/badge/Aladin-gray.svg?logo=aladin&logoColor=purple&logoWidth=20)](https://aladin.cds.unistra.fr/aladin.gml "https://aladin.cds.unistra.fr/aladin.gml") Lite widget, 
3. find and download a catalogue from [![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR"),
4. overlay the sources of this catalog on the sky view in the Aladin Lite widget, 
5. cross-match the sources with a large catalogue using [![Xmatch](https://custom-icon-badges.demolab.com/badge/Xmatch-gray.svg?logo=xmatch&logoColor=blue&logoWidth=20)](http://cdsxmatch.u-strasbg.fr/ "http://cdsxmatch.u-strasbg.fr/"). 

In [1]:
# Astronomy tools
import astropy.units as u

# Access astronomical databases
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch

# Sky visualization
import ipyaladin.aladin_widget as ipyal

## Step #1: Gather information on NGC4038 


[![Simbad](https://custom-icon-badges.demolab.com/badge/Simbad-gray.svg?logo=simbad&logoColor=lightblue&logoWidth=20)](https://simbad.u-strasbg.fr/simbad/ "https://simbad.u-strasbg.fr/simbad/")


As a first step, we check what information is directly available from the <ins>Simbad</ins> python module. We can look at the methods available for the `Simbad` class with the `help()` command:

In [2]:
help(Simbad)

Help on SimbadClass in module astroquery.simbad.core object:

class SimbadClass(SimbadBaseQuery)
 |  The class for querying the Simbad web service.
 |  
 |  Note that SIMBAD suggests submitting no more than 6 queries per second; if
 |  you submit more than that, your IP may be temporarily blacklisted
 |  (http://simbad.u-strasbg.fr/simbad/sim-help?Page=sim-url)
 |  
 |  Method resolution order:
 |      SimbadClass
 |      SimbadBaseQuery
 |      astroquery.query.BaseQuery
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  add_votable_fields(self, *args)
 |      Sets fields to be fetched in the VOTable. Must be one of those listed
 |      by `list_votable_fields`.
 |      
 |      Parameters
 |      ----------
 |      list of field_names
 |  
 |  get_field_description(self, field_name)
 |      Displays a description of the VOTable field.
 |      
 |      Parameters
 |      -------

Feel free to explore these methods. But in the next cell, let's print the results of `list_votable_fields`

In [3]:
print(Simbad.list_votable_fields())

--NOTES--

1. The parameter filtername must correspond to an existing filter. Filters include: B,V,R,I,J,K.  They are checked by SIMBAD but not astroquery.simbad

2. Fields beginning with rvz display the data as it is in the database. Fields beginning with rv force the display as a radial velocity. Fields beginning with z force the display as a redshift

3. For each measurement catalog, the VOTable contains all fields of the first measurement. When applicable, the first measurement is the mean one. 

Available VOTABLE fields:

bibcodelist(y1-y2)
biblio
cel
cl.g
coo(opt)
coo_bibcode
coo_err_angle
coo_err_maja
coo_err_mina
coo_qual
coo_wavelength
coordinates
dec(opt)
dec_prec
diameter
dim
dim_angle
dim_bibcode
dim_incl
dim_majaxis
dim_minaxis
dim_qual
dim_wavelength
dimensions
distance
distance_result
einstein
fe_h
flux(filtername)
flux_bibcode(filtername)
flux_error(filtername)
flux_name(filtername)
flux_qual(filtername)
flux_system(filtername)
flux_unit(filtername)
fluxdata(filtername)

We get a list of the available fields we could query to obtain in a table. If we are uncertain about the meaning of a field, we can get a description with `Simbad.get_field_description('field_name')`.

Scrolling the output of the cell above we also learn that the currently active fields are the main_id and the coordinates. If we want more information, we need to request it specifically. We can do so by running `Simbad.add_votable_fields('field_name')` before querying the object:

In [4]:
Simbad.get_field_description("biblio")

all bibcodes of an astronomical object separated with a ipipe


In [5]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields("otype", "dim", "dim_bibcode", "biblio")
info_simbad = Simbad.query_object("NGC4038")
info_simbad

<Table length=1>
 MAIN_ID       RA      ... SCRIPT_NUMBER_ID
            "h:m:s"    ...                 
  object     str13     ...      int32      
--------- ------------ ... ----------------
NGC  4038 12 01 53.002 ...                1

We now know the object type "galaxy in pair" `GinPair`, the object dimensions (major axis `GALDIM_MAJAXIS`, minor axis `GALDIM_MINAXIS`, inclination angle `GALDIM_ANGLE`) with the related bibliographic reference.

Let's search for linked objects: we query sources in a circle of 2 arcmin around NGC4038 of the same object type to find the other galaxy of the pair; then we query sources in the same area with 100% membership probability. 

In [6]:
siblings = Simbad.query_criteria("region(circle, NGC4038, 2m)", otype="GinPair")
siblings

<Table length=2>
 MAIN_ID       RA      ... SCRIPT_NUMBER_ID
            "h:m:s"    ...                 
  object     str13     ...      int32      
--------- ------------ ... ----------------
NGC  4038 12 01 53.002 ...                0
NGC  4039   12 01 53.8 ...                0

In [7]:
members = Simbad.query_criteria("region(circle, NGC4038, 2m)", membership=100)
members

<Table length=4>
      MAIN_ID            RA      ... SCRIPT_NUMBER_ID
                      "h:m:s"    ...                 
       object          str13     ...      int32      
------------------- ------------ ... ----------------
      NAME Antennae 12 01 53.170 ...                0
[CEB2007] Cluster 3  12 01 54.56 ...                0
          NGC  4038 12 01 53.002 ...                0
          NGC  4039   12 01 53.8 ...                0

You can learn more on the features of the Simbad python module here: https://astroquery.readthedocs.io/en/latest/simbad/simbad.html

and on the Simbad query criteria: http://simbad.cds.unistra.fr/simbad/sim-fsam

## Step #2: Visualise the source on a sky map

[![Aladin](https://custom-icon-badges.demolab.com/badge/Aladin-gray.svg?logo=aladin&logoColor=purple&logoWidth=20)](https://aladin.cds.unistra.fr/aladin.gml "https://aladin.cds.unistra.fr/aladin.gml")

We now open the [Aladin Lite widget](https://github.com/cds-astro/ipyaladin "https://github.com/cds-astro/ipyaladin"). We set it first to **center** on the Antennae galaxies, have a **field of view** of 0.5 $^\circ$ and show colour images from the Digitized Sky Survey ([DSS](https://archive.stsci.edu/cgi-bin/dss_form)). Typing `aladin` in the second line of code tells the notebook to display the widget. 

In [8]:
aladin = ipyal.Aladin(target="Antennae", fov=0.5, survey="P/DSS2/color")
aladin

Aladin(fov=0.5, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'ov…

As with any Aladin Lite implementation, you can interact with this widget. Try to:
 - scroll with your mouse pointer on the image to zoom in and out,  
 - select other image surveys and manage the current view with ![layers](Data/images/ipyaladin_layer.png)
 - look at another target using the search field ![search](Data/images/ipyaladin_search.png). 

You can also directly change the properties of the variable `aladin`. For example, to focus on M101 instead of the Antennae galaxies:

In [9]:
aladin.target = "M101"

Feel free to explore other targets and methods available for the Aladin widget.

## Step #3: Find and visualize additional data

[![Aladin](https://custom-icon-badges.demolab.com/badge/Aladin-gray.svg?logo=aladin&logoColor=purple&logoWidth=20)](https://aladin.cds.unistra.fr/aladin.gml "https://aladin.cds.unistra.fr/aladin.gml")
[![Simbad](https://custom-icon-badges.demolab.com/badge/Simbad-gray.svg?logo=simbad&logoColor=lightblue&logoWidth=20)](https://simbad.u-strasbg.fr/simbad/ "https://simbad.u-strasbg.fr/simbad/")
[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

We want to look for data related to HII regions in these interacting galaxies. We make a query in <ins>Simbad</ins> for HII regions within 14 arcmins around the Antennae galaxies. 

In [21]:
table_simbad = Simbad.query_criteria("region(circle, Antennae, 14m)", maintype="HII")
table_simbad.show_in_notebook(display_length=15)
# most browsers will open a scrolling window, rendering display_length useless but it is here
# to prevent some of us to have to scroll down the 317 lines of the table.

idx,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,OTYPE,GALDIM_MAJAXIS,GALDIM_MINAXIS,GALDIM_ANGLE,GALDIM_BIBCODE,BIBLIO,SCRIPT_NUMBER_ID
,,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,,arcmin,arcmin,deg,,,
0,RX J120155.4-185209,12 01 55.4,-18 52 09,5,5,--,--,0,D,,,HIIReg,--,--,0,,1997ApJ...478..542F|1995MNRAS.277..397R,0
1,RX J120150.8-185220,12 01 50.9,-18 52 20,5,5,--,--,0,D,,,HIIReg,--,--,0,,2000AJ....120..670N|1997ApJ...478..542F|1995MNRAS.277..397R,0
2,[MDL92] I,12 01 27.29,-19 00 54.7,6,6,--,--,0,D,,2004AJ....127..660S,HIIReg,--,--,0,,2004AJ....127..660S|2001AJ....122.2969H|1992A&A...256L..19M,0
3,[MDL92] II,12 01 27.57,-19 01 01.6,6,6,--,--,0,D,,2004AJ....127..660S,HIIReg,--,--,0,,2004AJ....127..660S|2001AJ....122.2969H|1992A&A...256L..19M,0
4,[MDL92] III,12 01 25.30,-19 00 40.3,6,6,--,--,0,D,,2004AJ....127..660S,HIIReg,--,--,0,,2004AJ....127..660S|2001AJ....122.2969H|1992A&A...256L..19M,0
5,[ZBF2015] Arp244 1,12 01 54.60,-18 53 03.1,6,6,--,--,0,D,O,2014MNRAS.445.1412Z,HIIReg,0.0875,0.0875,90,2014MNRAS.445.1412Z,2015MNRAS.451.1307Z|2014MNRAS.445.1412Z,0
6,[ZBF2015] Arp244 2,12 01 55.70,-18 52 42.1,6,6,--,--,0,D,O,2014MNRAS.445.1412Z,HIIReg,0.079,0.079,90,2014MNRAS.445.1412Z,2015MNRAS.451.1307Z|2014MNRAS.445.1412Z,0
7,[ZBF2015] Arp244 3,12 01 55.40,-18 52 48.5,6,6,--,--,0,D,O,2014MNRAS.445.1412Z,HIIReg,0.0585,0.0585,90,2014MNRAS.445.1412Z,2015MNRAS.451.1307Z|2014MNRAS.445.1412Z,0
8,[ZBF2015] Arp244 4,12 01 54.50,-18 51 55.9,6,6,--,--,0,D,O,2014MNRAS.445.1412Z,HIIReg,0.0878,0.0878,90,2014MNRAS.445.1412Z,2015MNRAS.451.1307Z|2014MNRAS.445.1412Z,0


Once we have identified a bibliographic reference of interest, we can search the corresponding catalogs on <ins>VizieR</ins>. Here, the Biblio column has quite some occurrences of the reference `'2015MNRAS.451.1307Z'`.

Remember that by default VizieR will only return 50 entries (as in the web interface). To get all sources, set `ROW_LIMIT=-1`.

A good practice is to check first what data are available by querying only a subsample, then customize your Vizier query with the useful rows and columns before downloading the entire catalog.

In [11]:
cat_viz = Vizier(row_limit=10).get_catalogs("2015MNRAS.451.1307Z")
cat_viz

TableList with 2 tables:
	'0:J/MNRAS/451/1307/HIIreg' with 15 column(s) and 10 row(s) 
	'1:J/MNRAS/451/1307/list' with 13 column(s) and 10 row(s) 

In the example above, we see that 2 tables are associated with the reference 2015MNRAS.451.1307Z: a table called 'HIIreg' accessible by the index [0] and a table called 'list' accessible by the index [1]. The number of rows has been fixed by the parameter `row_limit=10`.

Let's check the 'HIIreg' table:

In [12]:
print(cat_viz[0].meta["description"])

cat_viz[0].show_in_notebook()

Physical properties of the brightest HII regions in interacting galaxies (table A1) and isolated galaxies (table A2)


idx,Seq,n_Name,Name,Nf,RAJ2000,DEJ2000,logLHa,R,logMHII,rhoHII,sigma,alpVir,EWHa,logAge,Simbad
,,,,,'h:m:s','d:m:s',log(1e-07W),pc,log(solMass),solMass / pc3,km / s,,0.1 nm,log(yr),
0,1,int,NGC2146,3,06 18 39.4,+78 21 18.2,39.040,114,5.90,0.1270,15.0,40.0,159.79,6.770,Simbad
1,2,int,NGC2146,3,06 18 38.1,+78 21 26.8,38.830,116,5.80,0.0970,17.0,60.0,54.70,6.810,Simbad
2,3,int,NGC2146,3,06 18 44.6,+78 21 50.6,38.590,142,5.80,0.0550,11.0,30.0,125.30,6.780,Simbad
3,4,int,NGC2146,3,06 18 27.7,+78 22 21.0,38.470,155,5.81,0.0410,8.6,20.0,119.64,6.780,Simbad
4,5,int,NGC2146,3,06 18 35.0,+78 21 28.0,38.440,124,5.65,0.0560,15.0,80.0,103.35,6.790,Simbad
5,6,int,NGC2146,3,06 18 42.8,+78 21 58.1,38.410,136,5.70,0.0470,4.0,--,155.86,6.770,Simbad
6,7,int,NGC2146,3,06 18 39.8,+78 21 22.5,38.350,81,5.30,0.0960,15.0,100.0,109.47,6.790,Simbad
7,8,int,NGC2146,3,06 18 48.0,+78 21 43.3,38.320,144,5.69,0.0390,4.0,--,81.89,6.800,Simbad
8,9,int,NGC2146,3,06 18 42.1,+78 21 12.0,38.310,94,5.40,0.0730,4.0,--,103.95,6.790,Simbad


As for every function in python, we can call `Vizier?` in a code cell and run it to print the documentation. Here we can directly read that `columns` should be a list, `column_filters` a dictionary, and `row_limits` is an int for which -1 means infinity.

In [13]:
help(Vizier)

Help on VizierClass in module astroquery.vizier.core object:

class VizierClass(astroquery.query.BaseQuery)
 |  VizierClass(columns=['*'], column_filters={}, catalog=None, keywords=None, ucd='', timeout=60, vizier_server='vizier.u-strasbg.fr', row_limit=50)
 |  
 |  Method resolution order:
 |      VizierClass
 |      astroquery.query.BaseQuery
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, columns=['*'], column_filters={}, catalog=None, keywords=None, ucd='', timeout=60, vizier_server='vizier.u-strasbg.fr', row_limit=50)
 |      Parameters
 |      ----------
 |      columns : list
 |          List of strings
 |      column_filters : dict
 |      catalog : str or None
 |      keywords : str or None
 |      ucd : string
 |          "Unified Content Description" column descriptions.  Specifying
 |          these will select only catalogs that have columns matching the
 |          column descriptions defined on the Vizier web pages.
 |          See http://v

Now we can customize our VizieR query by selecting columns, filtering over the interacting galaxies (`n_Name='int'`), and keeping only the first region for each galaxy (`Seq=1`, for visualization purposes).

In [14]:
myViz = Vizier(
    columns=["Name", "RAJ2000", "DEJ2000", "logLHa", "R", "sigma"],
    column_filters={"Seq": "==1", "n_Name": "int"},
    row_limit=-1,
)

cat_hii = myViz.get_catalogs("2015MNRAS.451.1307Z")[0]
cat_hii.show_in_notebook()

idx,Name,RAJ2000,DEJ2000,logLHa,R,sigma
,,'h:m:s','d:m:s',log(1e-07W),pc,km / s
0,NGC2146,06 18 39.4,+78 21 18.2,39.040,114,15.0
1,Arp244,12 01 54.6,-18 53 03.1,40.200,280,32.8
2,NGC520,01 24 33.3,+03 48 04.6,39.120,255,14.1
3,UGC3995,07 44 09.2,+29 14 51.3,40.290,270,43.0
4,NGC3788,11 39 41.4,+31 54 23.3,39.700,280,11.9
5,NGC3786,11 39 45.1,+31 55 46.3,39.860,300,16.5
6,NGC2782,09 14 05.3,+40 06 50.4,39.800,166,28.0
7,NGC2993,09 45 48.3,-14 22 05.9,40.070,258,12.0
8,NGC2992,09 45 41.9,-14 19 33.4,39.180,100,16.0


We can also display the table on <ins>Aladin</ins>. 
This should work by simply typing `aladin.add_table(cat_hii)`
but if you get errors in the format of some columns, 
you can just send the name and coordinates to Aladin:

In [15]:
aladin.add_table(cat_hii["Name", "RAJ2000", "DEJ2000"])

Go up to the <ins>Aladin Lite</ins> widget, zoom out to see all the sources, then rotate and zoom in to see the interacting galaxies. You can click on one source to display the information in the table. 

Change the view to the AllWISE Infrared survey, either in the widget:

`Base image layer -> The Wide-field Infrared Survey Explorer` 

or in the command line below.

You can find the list of available sky maps at https://aladin.u-strasbg.fr/hips/list

In [16]:
aladin.survey = "P/allWISE/color"

If you want to overlay the survey instead of replacing it:

`aladin.overlay_survey= 'P/allWISE/color'`

`aladin.overlay_survey_opacity = 0.5`

**Exercise:**

Retrieve the second table from Zaragoza-Cardiel+, 2015 (2015MNRAS.451.1307Z) containing data related to the image fits files and try to answer the questions:
- What are the typical sizes of the fits images?
- What is the approximate resolution of the images? [arcsec/pix]

In [17]:
# write your code here

## Step #5: Cross-match tables

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")
[![Xmatch](https://custom-icon-badges.demolab.com/badge/Xmatch-gray.svg?logo=xmatch&logoColor=blue&logoWidth=20)](http://cdsxmatch.u-strasbg.fr/ "http://cdsxmatch.u-strasbg.fr/")

We have HII data for a sample of interacting galaxies. Since we are also interested in the infrared fluxes we search for the Wide-field Infrared Survey Explorer ([AllWISE](https://wise2.ipac.caltech.edu/docs/release/allwise/)) data release. 

We want to match the sources in J/MNRAS/451/1307/HIIreg with the AllWISE source catalog that contains 747 million objects!

We use the <ins>X-Match</ins> python module for cross-identification of objects. It is particularly efficient and fast with very large catalogs, like AllWISE. All we need are the names of the catalogs, the names of the columns containing the coordinates of the sources, and the desired maximum distance for the match (optionally the area as well, otherwise all-sky is the default).

In [18]:
# What is the AllWISE catalogue reference in VizieR?
cat_allwise = Vizier.get_catalogs("AllWISE")
cat_allwise

TableList with 1 tables:
	'0:II/328/allwise' with 28 column(s) and 50 row(s) 

In [19]:
xmatch_zbf_wise = XMatch.query(
    cat1="vizier:J/MNRAS/451/1307/HIIreg",
    cat2="vizier:II/328/allwise",
    max_distance=10 * u.arcsec,
    colRA1="RAJ2000",
    colDec1="DEJ2000",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
)
xmatch_zbf_wise.show_in_notebook()

idx,angDist,_RAJ2000,_DEJ2000,recno,Seq,n_Name,Name,Nf,RAJ2000_1,DEJ2000_1,logLHa,R,logMHII,rhoHII,sigma,alpVir,EWHa,logAge,Simbad,AllWISE,RAJ2000_2,DEJ2000_2,eeMaj,eeMin,eePA,W1mag,W2mag,W3mag,W4mag,Jmag,Hmag,Kmag,e_W1mag,e_W2mag,e_W3mag,e_W4mag,e_Jmag,e_Hmag,e_Kmag,ID,ccf,ex,var,qph,pmRA,e_pmRA,pmDE,e_pmDE,d2M
0,9.439952,250.775,36.839056,1820,27,iso,NGC6207,0,16 43 06.0,+36 50 20.6,37.69,66,4.9,0.06,4.0,--,134.26,6.78,Simbad,J164305.83+365029.8,250.7743217,36.8416214,0.0374,0.0369,67.0,12.87,12.918,8.29,5.588,--,--,--,0.073,0.025,0.022,0.037,--,--,--,2499136301351044678,0000,3,3000,AAAA,10,40,359,42,--
1,6.754932,250.767917,36.834944,1821,28,iso,NGC6207,0,16 43 04.3,+36 50 05.8,37.66,101,5.1,0.031,4.0,--,22.27,6.91,Simbad,J164303.76+365003.8,250.7656686,36.8344125,0.0212,0.0201,135.0,10.792,11.24,7.028,4.232,12.866,12.603,12.523,0.03,0.019,0.013,0.021,0.043,0.056,0.058,2499136301351044679,hd00,3,n9n3,AAAA,-929,25,-1205,26,0.458
2,8.575258,250.767917,36.834944,1821,28,iso,NGC6207,0,16 43 04.3,+36 50 05.8,37.66,101,5.1,0.031,4.0,--,22.27,6.91,Simbad,J164303.79+364959.7,250.7658199,36.8332538,0.0379,0.0338,158.6,10.953,10.936,7.758,5.483,--,--,--,0.023,0.022,0.024,0.066,--,--,--,2499136301351044647,d0d0,5,3200,AAAA,113,31,484,32,--
3,5.99581,250.76375,36.833083,1822,29,iso,NGC6207,0,16 43 03.3,+36 49 59.1,37.66,87,5.0,0.039,4.0,--,20.6,6.93,Simbad,J164303.79+364959.7,250.7658199,36.8332538,0.0379,0.0338,158.6,10.953,10.936,7.758,5.483,--,--,--,0.023,0.022,0.024,0.066,--,--,--,2499136301351044647,d0d0,5,3200,AAAA,113,31,484,32,--
4,7.312318,250.76375,36.833083,1822,29,iso,NGC6207,0,16 43 03.3,+36 49 59.1,37.66,87,5.0,0.039,4.0,--,20.6,6.93,Simbad,J164303.76+365003.8,250.7656686,36.8344125,0.0212,0.0201,135.0,10.792,11.24,7.028,4.232,12.866,12.603,12.523,0.03,0.019,0.013,0.021,0.043,0.056,0.058,2499136301351044679,hd00,3,n9n3,AAAA,-929,25,-1205,26,0.458
5,3.802161,250.766667,36.832444,1823,30,iso,NGC6207,0,16 43 04.0,+36 49 56.8,37.66,91,5.1,0.036,4.0,--,8.9,7.09,Simbad,J164303.79+364959.7,250.7658199,36.8332538,0.0379,0.0338,158.6,10.953,10.936,7.758,5.483,--,--,--,0.023,0.022,0.024,0.066,--,--,--,2499136301351044647,d0d0,5,3200,AAAA,113,31,484,32,--
6,7.648264,250.766667,36.832444,1823,30,iso,NGC6207,0,16 43 04.0,+36 49 56.8,37.66,91,5.1,0.036,4.0,--,8.9,7.09,Simbad,J164303.76+365003.8,250.7656686,36.8344125,0.0212,0.0201,135.0,10.792,11.24,7.028,4.232,12.866,12.603,12.523,0.03,0.019,0.013,0.021,0.043,0.056,0.058,2499136301351044679,hd00,3,n9n3,AAAA,-929,25,-1205,26,0.458
7,9.762527,250.769167,36.832833,1825,34,iso,NGC6207,0,16 43 04.6,+36 49 58.2,37.49,57,4.7,0.06,4.0,--,18.26,6.98,Simbad,J164303.79+364959.7,250.7658199,36.8332538,0.0379,0.0338,158.6,10.953,10.936,7.758,5.483,--,--,--,0.023,0.022,0.024,0.066,--,--,--,2499136301351044647,d0d0,5,3200,AAAA,113,31,484,32,--
8,4.218263,250.775,36.840583,1830,41,iso,NGC6207,0,16 43 06.0,+36 50 26.1,37.26,33,4.2,0.1,4.0,--,255.83,6.75,Simbad,J164305.83+365029.8,250.7743217,36.8416214,0.0374,0.0369,67.0,12.87,12.918,8.29,5.588,--,--,--,0.073,0.025,0.022,0.037,--,--,--,2499136301351044678,0000,3,3000,AAAA,10,40,359,42,--
9,0.814803,40.89875,1.377222,1834,1,iso,NGC1073,0,02 43 35.7,+01 22 38.0,39.22,124,6.04,0.138,13.4,23.0,522.67,6.59,Simbad,J024335.67+012237.2,40.8986511,1.3770184,0.0807,0.0757,126.3,14.868,14.664,10.041,7.24,--,--,--,0.033,0.065,0.066,0.129,--,--,--,408101501351028660,0000,3,nnnn,AAAB,--,--,--,--,--


<h4 align="center">This is the end of the tutorial.</h4> 

<h4 align="center">Try to apply what you have learned here to your data sets and science cases! </h4>